In [9]:
import subprocess
import os

reference = "/mnt/jupiter/johnsonlab/Capstone_proj/annotation_jasmine-the2899-mb-hirise-ahy4t__02-20-2022__hic_output.fasta"
out_dir = "/mnt/jupiter/johnsonlab/Capstone_proj/results/BQSR"
bam_dir = "/mnt/jupiter/johnsonlab/Capstone_proj/results/BAM/Sorted_RG"
genome_file = "/mnt/jupiter/johnsonlab/Capstone_proj/genome.txt"
gatk_path = "/bin/gatk-4.6.2.0/gatk"
threads = "30"

os.makedirs(out_dir, exist_ok=True)

files = ["10KP_RG"]
#"14KP_RG", "15AZ_RG", "28KP_RG", "31SMW_RG", "32SMW_RG", "34SMW_RG", "35SMW_RG", "36SMW_RG", "38SMW_RG", "39SMW_RG",
#"46SMW_RG", "52SMW_RG", "55SMW_RG", "57SMW_RG", "69SMF_RG", "70SMF_RG", "8KP_RG", "9KP_RG", "MOR021_RG", "MOR023_RG"

for file in files:
    print(f"🔄 Processing sample: {file}")

    sorted_rg = f"{bam_dir}/{file}.sorted.bam"
    raw_vcf = f"{out_dir}/{file}.raw.vcf"
    filtered_vcf = f"{out_dir}/{file}.filtered.vcf"
    passonly_vcf = f"{out_dir}/{file}.passonly.vcf"
    recal_table = f"{out_dir}/{file}.recal.table"
    recal_bam = f"{out_dir}/{file}.recalibrated.bam"
    final_gvcf = f"{out_dir}/{file}.final.g.vcf"
    joint_vcf = f"{out_dir}/{file}.joint_genotyped.vcf"
    snps_vcf = f"{out_dir}/{file}.joint_snps.vcf"
    maf_filtered_vcf = f"{out_dir}/{file}.selected_snps.vcf"
    snp_bed = f"{out_dir}/{file}.snps.bed"
    snp_windows = f"{out_dir}/{file}.snp_windows_30bp.bed"
    snp_counts = f"{out_dir}/{file}.snps_with_counts.txt"
    final_snps_bed = f"{out_dir}/{file}.final_snps.bed"

    # === Step 1: Initial Variant Calling ===
    subprocess.run([
        gatk_path, "HaplotypeCaller",
        "-R", reference,
        "-I", sorted_rg,
        "-O", raw_vcf,
        "--native-pair-hmm-threads", threads
    ])

    # === Step 2: Hard Filtering ===
    subprocess.run([
        gatk_path, "VariantFiltration",
        "-V", raw_vcf,
        "--filter-expression", "QD < 2.0 || FS > 60.0 || MQ < 40.0",
        "--filter-name", "FAIL",
        "-O", filtered_vcf
    ])
    subprocess.run([
        gatk_path, "SelectVariants",
        "-V", filtered_vcf,
        "--exclude-filtered",
        "-O", passonly_vcf
    ])

    # === Step 3: BQSR ===
    subprocess.run([
        gatk_path, "BaseRecalibrator",
        "-I", sorted_rg,
        "-R", reference,
        "--known-sites", passonly_vcf,
        "-O", recal_table,
        "--num-threads", threads
    ])
    subprocess.run([
        gatk_path, "ApplyBQSR",
        "-R", reference,
        "-I", sorted_rg,
        "--bqsr-recal-file", recal_table,
        "-O", recal_bam,
        "--num-threads", threads
    ])

    # === Step 4: Final Variant Calling ===
    subprocess.run([
        gatk_path, "HaplotypeCaller",
        "-R", reference,
        "-I", recal_bam,
        "-O", final_gvcf,
        "-ERC", "GVCF",
        "--native-pair-hmm-threads", threads
    ])

    # === Step 5: Joint Genotyping (single-sample placeholder) ===
    subprocess.run([
        gatk_path, "GenotypeGVCFs",
        "-R", reference,
        "-V", final_gvcf,
        "-O", joint_vcf
    ])

    # === Step 6: Select SNPs only ===
    subprocess.run([
        gatk_path, "SelectVariants",
        "-V", joint_vcf,
        "--select-type-to-include", "SNP",
        "-O", snps_vcf
    ])

    '''# === Step 7: MAF + missing data filtering ===
    subprocess.run([
        "bcftools", "view",
        "-i", "MAF>0.1 && F_MISSING<0.05",
        "-o", maf_filtered_vcf,
        snps_vcf
        #this doesn't work because there are no MAFs or F_MISSING added to the vcfs yet
        #MAF: Minor Allele Frequency
        #F_MISSING: Fraction of Missing Genotypes
    ])'''
    # === Step 7: Annotate with MAF and F_MISSING, then filter ===
    annotated_vcf = f"{out_dir}/{file}.annotated_snps.vcf"

    # Step 7a: Annotate SNPs VCF with MAF and F_MISSING
    with open(annotated_vcf, 'w') as annot_out:
        fill_tags = subprocess.Popen([
            "bcftools", "+fill-tags", snps_vcf, "-Ou", "--", "-t", "MAF,F_MISSING"
        ], stdout=subprocess.PIPE)
        subprocess.run([
            "bcftools", "view", "-Ov"
        ], stdin=fill_tags.stdout, stdout=annot_out)

    # Step 7b: Filter based on MAF > 0.1 and F_MISSING < 0.05
    with open(maf_filtered_vcf, 'w') as filt_out:
        subprocess.run([
            "bcftools", "view",
            "-i", "MAF>0.1 && F_MISSING<0.05",
            annotated_vcf
        ], stdout=filt_out)

    # === Step 8: Convert SNPs to BED ===
    with open(snp_bed, 'w') as bed_out:
        subprocess.run([
            "bcftools", "query",
            "-f", "%CHROM\t%POS0\t%POS\t%ID\n",
            maf_filtered_vcf
        ], stdout=bed_out)

    # === Step 9: Create 30bp windows ===
    with open(snp_windows, 'w') as out_win:
        subprocess.run([
            "bedtools", "slop",
            "-i", snp_bed,
            "-g", genome_file,
            "-b", "30"
        ], stdout=out_win)

    # === Step 10: Count SNPs per window ===
    with open(snp_counts, 'w') as out_count:
        subprocess.run([
            "bedtools", "intersect",
            "-a", snp_windows,
            "-b", snp_bed,
            "-c"
        ], stdout=out_count)

    # === Step 11: Filter SNPs without nearby neighbors ===
    with open(snp_counts) as infile, open(final_snps_bed, 'w') as outfile:
        for line in infile:
            fields = line.strip().split('\t')
            if int(fields[4]) == 1:
                outfile.write('\t'.join(fields[:3]) + '\n')

    print(f"✅ Finished processing sample: {file}")

🔄 Processing sample: 10KP_RG


07:11:47.468 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/usr/bin/gatk-4.6.2.0/gatk-package-4.6.2.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
07:11:47.673 INFO  HaplotypeCaller - ------------------------------------------------------------
07:11:47.676 INFO  HaplotypeCaller - The Genome Analysis Toolkit (GATK) v4.6.2.0
07:11:47.676 INFO  HaplotypeCaller - For support and documentation go to https://software.broadinstitute.org/gatk/
07:11:47.676 INFO  HaplotypeCaller - Executing as chdahl@genomics1 on Linux v5.4.0-214-generic amd64
07:11:47.676 INFO  HaplotypeCaller - Java runtime: OpenJDK 64-Bit Server VM v17.0.15+6-Ubuntu-0ubuntu120.04
07:11:47.676 INFO  HaplotypeCaller - Start Date/Time: May 7, 2025 at 7:11:47 AM UTC
07:11:47.677 INFO  HaplotypeCaller - ------------------------------------------------------------
07:11:47.677 INFO  HaplotypeCaller - ------------------------------------------------------------
07:11:47.677 INFO  HaplotypeCaller 

KeyboardInterrupt: 

In [ ]:
#PIPELINE
sorted.bam 
   ↓
First HaplotypeCaller → raw.vcf 
   ↓
Hard Filtering → passonly.vcf 
   ↓
BaseRecalibrator + ApplyBQSR → recalibrated.bam 
   ↓
Second HaplotypeCaller → final.g.vcf 
   ↓
CombineGVCFs + GenotypeGVCFs → joint.vcf 
   ↓
Filtering (MAF, call rate) → selected_snps.vcf 
   ↓
Design SNP array / GT-seq panel